<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_9_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import pickle
from skimage.feature import hog
import numpy as np
from collections import Counter
import random

In [ ]:
class CustomStandardScaler:
    def __init__(self):
        self.mean_ = None
        self.std_ = None

    def fit(self, X):
        self.mean_ = np.mean(X, axis=0)
        self.std_ = np.std(X, axis=0)
        self.std_[self.std_ == 0] = 1
        return self

    def transform(self, X):
        return (X - self.mean_) / self.std_

    def fit_transform(self, X):
        return self.fit(X).transform(X)

In [ ]:
class CustomKNNClassifier:
    def __init__(self, n_neighbors=3):
        self.k = n_neighbors
        self.X_train = None
        self.y_train = None
        self.classes_ = []

    def fit(self, X, y):
        self.X_train = np.array(X)
        self.y_train = np.array(y)
        self.classes_ = sorted(set(y))

    def predict(self, X):
        return [self._predict_single(x) for x in X]

    def predict_proba(self, X):
        proba_list = []
        for x in X:
            distances = np.linalg.norm(self.X_train - x, axis=1)
            nearest_indices = np.argsort(distances)[:self.k]
            nearest_labels = self.y_train[nearest_indices]
            counter = Counter(nearest_labels)
            total = sum(counter.values())
            proba = [counter.get(cls, 0) / total for cls in self.classes_]
            proba_list.append(proba)
        return np.array(proba_list)

    def score(self, X, y_true):
        y_pred = self.predict(X)
        correct = sum(yt == yp for yt, yp in zip(y_true, y_pred))
        return correct / len(y_true)

    def _predict_single(self, x):
        distances = np.linalg.norm(self.X_train - x, axis=1)
        nearest_indices = np.argsort(distances)[:self.k]
        nearest_labels = self.y_train[nearest_indices]
        counter = Counter(nearest_labels)
        return counter.most_common(1)[0][0]

In [ ]:
class ReadySymbolClassifier:
    def __init__(self, target_size=64):
        self.model = None
        self.scaler = None
        self.classes = []
        self.target_size = target_size

    def extract_features(self, img):
        # Приведение к одному размеру
        img = cv2.resize(img, (self.target_size, self.target_size), interpolation=cv2.INTER_AREA)

        # HOG-признаки
        hog_features = hog(img, pixels_per_cell=(20, 20),
                           cells_per_block=(2, 2),
                           visualize=False)

        # проекционные профили
        h_proj = np.sum(img, axis=1) / 255
        v_proj = np.sum(img, axis=0) / 255

        # средние значения по регионам (5×5)
        regions = []
        h, w = img.shape
        for i in range(5):
            for j in range(5):
                region = img[i * h // 5:(i + 1) * h // 5, j * w // 5:(j + 1) * w // 5]
                regions.append(np.mean(region))

        # моменты
        moments = cv2.moments(img)
        hu_moments = cv2.HuMoments(moments).flatten()

        features = np.concatenate([
            hog_features,
            h_proj,
            v_proj,
            regions,
            hu_moments
        ])

        return features

    def train(self, dataset_path):
        features = []
        labels = []

        print("Обучение на готовых данных")
        for class_name in os.listdir(dataset_path):
            class_dir = os.path.join(dataset_path, class_name)
            if not os.path.isdir(class_dir):
                continue

            print(f"Загрузка класса {class_name}...")
            for img_name in os.listdir(class_dir):
                if img_name.lower().endswith('.png'):
                    img_path = os.path.join(class_dir, img_name)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                    if img is not None:
                        features.append(self.extract_features(img))
                        labels.append(class_name)

        self.scaler = CustomStandardScaler()
        features = self.scaler.fit_transform(features)
        self.classes = sorted(list(set(labels)))

        self.model = CustomKNNClassifier(n_neighbors=5)
        self.model.fit(features, labels)

        train_acc = self.model.score(features, labels)
        print(f"Обучение завершено. Точность: {train_acc:.2%}")
        print(f"Классов: {len(self.classes)}, Признаков: {features.shape[1]}")

    def save_model(self, model_path):
        with open(model_path, 'wb') as f:
            pickle.dump({
                'model': self.model,
                'scaler': self.scaler,
                'classes': self.classes,
                'target_size': self.target_size
            }, f)
        print(f"Модель сохранена в {model_path}")

    def load_model(self, model_path):
        with open(model_path, 'rb') as f:
            data = pickle.load(f)
            self.model = data['model']
            self.scaler = data['scaler']
            self.classes = data['classes']
            self.target_size = data['target_size']
        print(f"Модель загружена. Классов: {len(self.classes)}")

    def recognize(self, img_path):
        if self.model is None:
            print("Ошибка: модель не загружена")
            return None

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print("Ошибка: не удалось загрузить изображение")
            return None

        features = self.extract_features(img)
        features = self.scaler.transform([features])

        probas = self.model.predict_proba(features)[0]
        top3 = np.argsort(probas)[-3:][::-1]

        results = []
        for i in top3:
            results.append({
                'symbol': self.model.classes_[i],
                'confidence': f"{probas[i]:.2%}"
            })

        return results

In [ ]:
def main(dataset_path, mode, target_sizes):
    classifier = ReadySymbolClassifier(target_size = target_sizes)
    if mode == 1:
        if not os.path.exists(dataset_path):
            print(f"Ошибка: папка {dataset_path} не найдена")
            return

        model_path = "classifier.pkl"
        classifier.train(dataset_path)
        classifier.save_model(model_path)

    elif mode == 2:
        model_path = "classifier.pkl"
        if not os.path.exists(model_path):
            print(f"Ошибка: файл модели {model_path} не найден")
            return
        if not os.path.exists(dataset_path):
            print(f"Ошибка: папка {dataset_path} не найдена")
            return

        classifier.load_model(model_path)

        correct = 0
        total = 0

        all_classes = [cls for cls in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, cls))]
        all_classes = sorted(all_classes)

        for cls in all_classes:
            cls_path = os.path.join(dataset_path, cls)
            all_images = [f for f in os.listdir(cls_path) if f.lower().endswith('.png')]
            selected_images = random.sample(all_images, min(10, len(all_images)))

            for img_name in selected_images:
                img_path = os.path.join(cls_path, img_name)
                results = classifier.recognize(img_path)

                if results:
                    predicted = results[0]['symbol']
                    print(f"{os.path.basename(img_name)}: распознан символ '{predicted}'")
                    if predicted == cls:
                        correct += 1
                    total += 1

        if total > 0:
            accuracy = 100 * correct / total
            print(f"\nРаспознано верно: {correct}/{total} изображений")
            print(f"Точность распознавания: {accuracy:.2f}%")
        else:
            print("Нет изображений для распознавания.")
    else:
        print("Неверный режим. Выберите 1 или 2.")


In [ ]:
main("processed_dataset/train", 1, 64)

Обучение на готовых данных
Загрузка класса 2...
Загрузка класса 5...
Загрузка класса Y...
Загрузка класса G...
Загрузка класса 4...
Загрузка класса R...
Загрузка класса N...
Загрузка класса P...
Загрузка класса 7...
Загрузка класса W...
Загрузка класса 9...
Загрузка класса F...
Загрузка класса 1...
Загрузка класса A...
Загрузка класса 6...
Загрузка класса X...
Загрузка класса 8...
Загрузка класса C...
Загрузка класса E...
Загрузка класса L...
Загрузка класса I...
Загрузка класса M...
Загрузка класса 10...
Загрузка класса D...
Загрузка класса 3...
Загрузка класса S...
Загрузка класса O...
Загрузка класса H...
Загрузка класса U...
Загрузка класса Z...
Загрузка класса 0...
Загрузка класса B...
Загрузка класса K...
Загрузка класса T...
Загрузка класса J...
Загрузка класса Q...
Загрузка класса V...
Обучение завершено. Точность: 84.87%
Классов: 37, Признаков: 304
Модель сохранена в classifier.pkl


In [ ]:
main("processed_dataset/test",2, 64)

Модель загружена. Классов: 37
0006.png: распознан символ '0'
0011.png: распознан символ 'O'
0022.png: распознан символ '0'
0027.png: распознан символ 'O'
0004.png: распознан символ '0'
0014.png: распознан символ 'Q'
0001.png: распознан символ '0'
0024.png: распознан символ '0'
0002.png: распознан символ '0'
0015.png: распознан символ 'O'
0024.png: распознан символ 'I'
0010.png: распознан символ '1'
0007.png: распознан символ '1'
0005.png: распознан символ 'I'
0022.png: распознан символ '1'
0018.png: распознан символ '1'
0028.png: распознан символ '1'
0029.png: распознан символ 'I'
0025.png: распознан символ '1'
0001.png: распознан символ '4'
0000.png: распознан символ '10'
0026.png: распознан символ '5'
0007.png: распознан символ '2'
0024.png: распознан символ '2'
0027.png: распознан символ '2'
0013.png: распознан символ '2'
0009.png: распознан символ '2'
0019.png: распознан символ '2'
0014.png: распознан символ 'X'
0004.png: распознан символ '2'
0029.png: распознан символ '2'
0016.png

In [ ]:
!unzip processed_dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: processed_dataset/train/X/0017.png  
  inflating: processed_dataset/train/X/0057.png  
  inflating: processed_dataset/train/X/0001.png  
  inflating: processed_dataset/train/X/0051.png  
 extracting: processed_dataset/train/X/0003.png  
  inflating: processed_dataset/train/X/0004.png  
  inflating: processed_dataset/train/X/0049.png  
  inflating: processed_dataset/train/X/0010.png  
  inflating: processed_dataset/train/X/0007.png  
  inflating: processed_dataset/train/X/0023.png  
  inflating: processed_dataset/train/X/0018.png  
 extracting: processed_dataset/train/X/0009.png  
  inflating: processed_dataset/train/X/0013.png  
  inflating: processed_dataset/train/X/0058.png  
  inflating: processed_dataset/train/X/0016.png  
  inflating: processed_dataset/train/X/0047.png  
  inflating: processed_dataset/train/X/0045.png  
  inflating: processed_dataset/train/X/0054.png  
  inflating: processed_dataset/t